In [2]:
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.initializers import Zeros
import sklearn.datasets as datasets
import pickle

2024-11-22 22:20:42.379042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732332042.528958    6541 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732332042.574257    6541 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 22:20:42.922202: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
CENTERS = [[0, 0]]
DATA, _ = datasets.make_blobs(
        n_samples=200,
        centers=CENTERS,
        cluster_std=1,
        random_state=1
    )

cw1 = -1 
cw2 = 1
cb = -1

sw1 = -1
sw2 = 1
sb = 1

X = np.array(list(filter(lambda x : cw1 * x[0] + cw2 * x[1] + cb < -.5 or cw1 * x[0] + cw2 * x[1] + cb > .5, DATA)))
X = np.array(list(filter(lambda x : sw1 * x[0] + sw2 * x[1] + sb < -.5 or sw1 * x[0] + sw2 * x[1] + sb > .5, X)))
C = np.array([0 if cw1 * x[0] + cw2 * x[1] + cb >= 0 else 1 for x in X])
S = np.array([0 if sw1 * x[0] + sw2 * x[1] + sb >= 0 else 1 for x in X])
pass_fail = C ^ S


In [16]:
# Build the model using Input layer
inputs = keras.layers.Input(shape=(2,))
hidden = keras.layers.Dense(3, activation="sigmoid")(inputs)  # Hidden layer with 3D output
outputs = keras.layers.Dense(1, activation="sigmoid")(hidden)  # Output layer
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-1), loss="binary_crossentropy")
# Prepare storage for hidden space data
hidden_space_trajectory = []
original_points = X  # Store the original points

hidden_layer_model = keras.Model(inputs=model.input, outputs=model.layers[1].output)

In [17]:
# Train the model for all epochs and record the hidden space every 10 epochs
epochs_per_interval = 10
total_epochs = 100
hidden_space_trajectory = []

for epoch in range(1, total_epochs + 1):
    # Train for one epoch at a time to monitor progress
    model.fit(X, pass_fail, epochs=1, verbose=1)

    # Record the hidden space trajectory every 10 epochs
    if epoch % epochs_per_interval == 0:
        # Inspect weights
        weights = model.layers[1].get_weights()[0]  # Weights of the hidden layer
        print(f"Weights after {epoch} epochs:\n{weights}")
        hidden_positions = hidden_layer_model.predict(X)
        hidden_space_trajectory.append(hidden_positions)
        print(f"Recorded hidden space at epoch {epoch}")

# Convert hidden_space_trajectory to numpy array for storage
hidden_space_trajectory = np.array(hidden_space_trajectory)
print(f"Hidden space trajectory shape: {hidden_space_trajectory.shape}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.7091
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6674  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6422
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6188
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5830  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5332
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4798
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4117  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3659  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3073  
Weights after 10 epochs:
[[ 2.277854   3.2275057  2.928841 ]
 [-1.9287009 -2.97115   -3.0104022]]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Recorded hidden space at epoch 10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2757 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2296  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2015  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1716 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1465
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

In [19]:
# Save the trajectory and original points to a pickle file
with open("hidden_space_and_points.pkl", "wb") as f:
    pickle.dump({"hidden_space_trajectory": hidden_space_trajectory, "original_points": original_points}, f)

print(f"Hidden space trajectory saved with shape: {hidden_space_trajectory.shape}")
print(f"Original points saved with shape: {original_points.shape}")

Hidden space trajectory saved with shape: (10, 115, 3)
Original points saved with shape: (115, 2)
